In [ ]:
!pip install kaggle gradio opencv-python-headless

In [ ]:
import os

os.makedirs("data", exist_ok=True)
os.environ["KAGGLE_USERNAME"] = "whysushe"
os.environ["KAGGLE_KEY"] = "kaggle"

!kaggle datasets download -d msambare/fer2013 -p ./data
!unzip -q ./data/fer2013.zip -d ./data
!ls ./data


Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
  0% 0.00/60.3M [00:00<?, ?B/s]
100% 60.3M/60.3M [00:00<00:00, 1.40GB/s]
fer2013.zip  test  train


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import pathlib

In [ ]:
data_dir = pathlib.Path("data")

train_dir = data_dir / "train"
test_dir = data_dir / "test"

img_height = 48
img_width = 48
batch_size = 64

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training",
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=False,
)

class_names = train_ds.class_names
num_classes = len(class_names)

print(class_names)
print(num_classes)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Found 28709 files belonging to 7 classes.
Using 5741 files for validation.
Found 7178 files belonging to 7 classes.
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
7


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

normalization_layer = layers.Rescaling(1.0 / 255.0)

def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = normalization_layer(image)
    return image, label

train_ds_prep = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE).cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds_prep = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE).cache().prefetch(AUTOTUNE)
test_ds_prep = test_ds.map(preprocess, num_parallel_calls=AUTOTUNE).cache().prefetch(AUTOTUNE)

In [ ]:
input_shape = (img_height, img_width, 1)

model = models.Sequential(
    [
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 24, 24, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 6, 6, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 6, 6, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 981,639 (3.74 MB)

 Trainable params: 980,679 (3.74 MB)

 Non-trainable params: 960 (3.75 KB)

In [ ]:
epochs = 25

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'emotions_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)


history = model.fit(
    train_ds_prep,
    validation_data=val_ds_prep,
    epochs=epochs,
)

Epoch 1/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 217s 578ms/step - accuracy: 0.2441 - loss: 2.1716 - val_accuracy: 0.2616 - val_loss: 1.8444
Epoch 2/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 257s 572ms/step - accuracy: 0.3689 - loss: 1.5925 - val_accuracy: 0.3912 - val_loss: 1.5473
Epoch 3/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 255s 553ms/step - accuracy: 0.4204 - loss: 1.4501 - val_accuracy: 0.4774 - val_loss: 1.3519
Epoch 4/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 202s 552ms/step - accuracy: 0.4543 - loss: 1.3708 - val_accuracy: 0.4781 - val_loss: 1.3461
Epoch 5/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 198s 552ms/step - accuracy: 0.4932 - loss: 1.2868 - val_accuracy: 0.4976 - val_loss: 1.2974
Epoch 6/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 200s 557ms/step - accuracy: 0.5235 - loss: 1.2185 - val_accuracy: 0.4841 - val_loss: 1.3216
Epoch 7/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 200s 552ms/step - accuracy: 0.5591 - loss: 1.1401 - val_accuracy: 0.5266 - val_loss: 1.2731
Epoch 8/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 196s 547ms/step - accuracy: 0.5911 -

In [ ]:
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml -O haarcascade_frontalface_default.xml


In [ ]:
test_loss, test_acc = model.evaluate(test_ds_prep)
print("Test accuracy:", test_acc)

113/113 ━━━━━━━━━━━━━━━━━━━━ 13s 116ms/step - accuracy: 0.5015 - loss: 3.0215
Test accuracy: 0.5295346975326538


In [ ]:
from tensorflow.keras.models import load_model
model.save("emotions_model.keras")

''' (to run later)
from tensorflow.keras.models import load_model

model = load_model("emotions_model.keras")
print("Model reloaded.")
print(class_names)
'''

In [ ]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

emoji_map = {
    "angry": "😠",
    "disgust": "🤢",
    "fear": "😨",
    "happy": "😄",
    "neutral": "😐",
    "sad": "😢",
    "surprise": "😲",
}

def preprocess_single_image(image):
    if image is None:
        return None
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if image.shape[2] == 3 else image
    gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(faces) > 0:
        x, y, w, h = max(faces, key=lambda b: b[2] * b[3])
        face_roi = gray[y:y+h, x:x+w]
    else:
        h, w = gray.shape
        size = min(h, w)
        y0 = (h - size) // 2
        x0 = (w - size) // 2
        face_roi = gray[y0:y0+size, x0:x0+size]
    face_roi = cv2.resize(face_roi, (img_width, img_height))
    face_roi = face_roi.astype("float32") / 255.0
    face_roi = np.expand_dims(face_roi, axis=-1)
    face_roi = np.expand_dims(face_roi, axis=0)
    return face_roi

def predict_emotion_from_image(image):
    x = preprocess_single_image(image)
    if x is None:
        return "no_face", ""
    preds = model.predict(x)
    idx = np.argmax(preds[0])
    label = class_names[idx]
    emoji = emoji_map.get(label.lower(), "")
    return label, emoji

In [ ]:
import gradio as gr

def gradio_infer(image):
    label, emoji = predict_emotion_from_image(image)
    return label, emoji

demo = gr.Interface(
    fn=gradio_infer,
    inputs=gr.Image(sources=["webcam", "upload"], type="numpy"),
    outputs=[gr.Label(), gr.Textbox(label="Emoji")],
    live=True,
    title="Facial Emotion Recognition with FER-2013",
    description="Real-time facial emotion recognition from webcam with matching emojis.",
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5dcb85bec863e5f992.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
